In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [2]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [3]:
AccidentsData = spark.read.format('csv').option('header', 'true').load('USAccidents.csv')
AccidentsData.createOrReplaceTempView('AccidentsDataDetails') 
AccidentsData.show()

+---------+-------+--------+----------------+----------------+------------------+------------+------------------+------------+--------+--------------------+----------------+-----------------+----------------+-----+----------+-------+-----------+------------+-----------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
|       ID| Source|Severity|      Start_Time|        End_Time|         Start_Lat|   Start_Lng|           End_Lat|     End_Lng|Distance|         Description|          Street|             City|          County|State|   Zipcode|Country|   Timezone|Airport_Code|Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(i

In [4]:
#Goal 1
spark.sql("SELECT State, COUNT(*) AS AccidentCount FROM AccidentsDataDetails GROUP BY State ORDER BY AccidentCount DESC").show()

+-----+-------------+
|State|AccidentCount|
+-----+-------------+
|   CA|        71784|
|   FL|        38371|
|   TX|        14338|
|   VA|        13088|
|   NY|        12666|
|   PA|        11729|
|   NC|        10149|
|   OR|         9924|
|   SC|         9406|
|   MN|         8397|
|   MD|         6390|
|   AZ|         6090|
|   NJ|         5801|
|   TN|         5349|
|   GA|         4940|
|   LA|         4690|
|   MI|         4524|
|   IL|         4315|
|   UT|         4180|
|   CO|         3105|
+-----+-------------+
only showing top 20 rows



In [5]:
#Goal 2:
spark.sql('Select Sunrise_Sunset, Civil_Twilight,Nautical_Twilight, Astronomical_Twilight, AVG(Severity) FROM AccidentsDataDetails GROUP BY Sunrise_Sunset, Civil_Twilight,Nautical_Twilight, Astronomical_Twilight, Severity').show()

+--------------+--------------+-----------------+---------------------+-------------+
|Sunrise_Sunset|Civil_Twilight|Nautical_Twilight|Astronomical_Twilight|avg(Severity)|
+--------------+--------------+-----------------+---------------------+-------------+
|         Night|           Day|              Day|                  Day|          4.0|
|         Night|         Night|              Day|                  Day|          4.0|
|           Day|           Day|              Day|                Night|          4.0|
|         Night|         Night|              Day|                  Day|          2.0|
|         Night|           Day|              Day|                  Day|          1.0|
|         Night|         Night|            Night|                  Day|          2.0|
|         Night|         Night|            Night|                  Day|          1.0|
|         Night|         Night|            Night|                  Day|          3.0|
|         Night|         Night|              Day|     

In [6]:
#Goal 3:
spark.sql(' select count(case when Amenity = "TRUE" then 1 end) as Amenity_Count,count(case when Bump = "TRUE" then 1 end) as Bump_Count,count(case when Crossing = "TRUE" then 1 end) as Crossing_Count,count(case when Junction = "TRUE" then 1 end) as Junction_Count,count(case when No_Exit = "TRUE" then 1 end) as No_Exit_Count,count(case when Railway = "TRUE" then 1 end) as Railway_Count,count(case when Roundabout = "TRUE" then 1 end) as Roundabout_Count,count(case when Station = "TRUE" then 1 end) as Station_Count,count(case when Stop = "TRUE" then 1 end) as Stop_Count,count(case when Traffic_Calming = "TRUE" then 1 end) as Traffic_Calming_Count,count(case when Traffic_Signal = "TRUE" then 1 end) as Traffic_Signal_Count,count(case when Turning_Loop = "TRUE" then 1 end) as Turning_Loop_Count from AccidentsDataDetails').show()

+-------------+----------+--------------+--------------+-------------+-------------+----------------+-------------+----------+---------------------+--------------------+------------------+
|Amenity_Count|Bump_Count|Crossing_Count|Junction_Count|No_Exit_Count|Railway_Count|Roundabout_Count|Station_Count|Stop_Count|Traffic_Calming_Count|Traffic_Signal_Count|Turning_Loop_Count|
+-------------+----------+--------------+--------------+-------------+-------------+----------------+-------------+----------+---------------------+--------------------+------------------+
|         3044|       137|         24717|         25531|          646|         2178|               8|         7191|      6996|                  260|               25261|                 0|
+-------------+----------+--------------+--------------+-------------+-------------+----------------+-------------+----------+---------------------+--------------------+------------------+



In [7]:
# Goal 4:
spark.sql("SELECT Weather_Condition, COUNT(*) AS AccidentCount FROM AccidentsDataDetails GROUP BY Weather_Condition ORDER BY AccidentCount DESC").show()

+--------------------+-------------+
|   Weather_Condition|AccidentCount|
+--------------------+-------------+
|                Fair|       116888|
|              Cloudy|        35716|
|       Mostly Cloudy|        34842|
|       Partly Cloudy|        24099|
|          Light Rain|        12198|
|               Clear|        11752|
|                NULL|         6926|
|            Overcast|         5807|
|          Light Snow|         5286|
|                 Fog|         3938|
|                Rain|         3035|
|    Scattered Clouds|         2960|
|                Haze|         2697|
|        Fair / Windy|         1796|
|          Heavy Rain|         1149|
|Thunder in the Vi...|          840|
|Mostly Cloudy / W...|          799|
|      Cloudy / Windy|          762|
|       Light Drizzle|          712|
|                Snow|          687|
+--------------------+-------------+
only showing top 20 rows



In [8]:
#Goal 5:
spark.sql("SELECT Sunrise_Sunset,COUNT(*) AS AccidentCount FROM AccidentsDataDetails GROUP BY Sunrise_Sunset ORDER BY AccidentCount DESC").show()

+--------------+-------------+
|Sunrise_Sunset|AccidentCount|
+--------------+-------------+
|           Day|       182222|
|         Night|        96022|
|          NULL|         1379|
+--------------+-------------+



In [9]:
#Goal 6:
spark.sql("SELECT Severity, COUNT(*) AS AccidentCount FROM AccidentsDataDetails GROUP BY Severity ORDER BY AccidentCount DESC").show()

+--------+-------------+
|Severity|AccidentCount|
+--------+-------------+
|       2|       255076|
|       4|        12129|
|       3|        10626|
|       1|         1792|
+--------+-------------+



In [10]:
#Goal 7:
spark.sql('select count(case when distance < 1 then 1 end) as short, count(case when distance between 1 and 5 then 1 end) as mid, count(case when distance > 5 then 1 end) as Long from AccidentsDataDetails').show()

+------+-----+----+
| short|  mid|Long|
+------+-----+----+
|217245|57574|4804|
+------+-----+----+



In [ ]:
#Goal 8:
